# Q3:  3. What beer (and category) has the maximum ABV (alcohol graduation)? and the minimum?


 **Reading the dataset (previously cleaned) about beers, breweries, and categories of beers.**

Beers

In [11]:
import pandas as pd
beers = pd.read_csv('REAL_Data_beers/NEW_DATA/beers_new.csv')
beers.head()

,brewery_id,name,cat_id,style_id,abv,ibu,srm,upc,filepath,descript,last_mod
0,812,Hocus Pocus,11,116,4.5,0,0,0,NaN,Our take on a classic summer ale. A toast to ...,2010-07-22 20:00:20
1,264,Grimbergen Blonde,1,-1,6.7,0,0,0,NaN,NaN,2010-07-22 20:00:20
2,779,Widdershins Barleywine,1,-1,9.1,0,0,0,NaN,NaN,2010-07-22 20:00:20
3,287,Lucifer,1,-1,8.5,0,0,0,NaN,NaN,2010-07-22 20:00:20
4,1056,Bitter,1,-1,4.0,0,0,0,NaN,NaN,2010-07-22 20:00:20


Breweries

In [5]:
breweries = pd.read_csv('REAL_Data_beers/NEW_DATA/brewery_new.csv')
breweries.head()

,id,name,address1,city,state,country,descript
0,1,(512) Brewing Company,"407 Radam, F200",Austin,Texas,United States,(512) Brewing Company is a microbrewery locate...
1,2,21st Amendment Brewery Cafe,563 Second Street,San Francisco,California,United States,The 21st Amendment Brewery offers a variety of...
2,3,3 Fonteinen Brouwerij Ambachtelijke Geuzestekerij,Hoogstraat 2A,Beersel,Vlaams Brabant,Belgium,NaN
3,4,Aass Brewery,Ole Steensgt. 10 Postboks 1530,Drammen,NaN,Norway,Aass Brewery was established in 1834 and is th...
4,5,Abbaye de Leffe,Dinant,Dinant,Namur,Belgium,NaN


Categories of beers

In [7]:
categories = pd.read_csv('REAL_Data_beers/NEW_DATA/categories_new.csv')
categories

,id,cat_name
0,1,British Ale
1,2,Irish Ale
2,3,North American Ale
3,4,German Ale
4,5,Belgian and French Ale
5,6,International Ale
6,7,German Lager
7,8,North American Lager
8,9,Other Lager
9,10,International Lager
